In [404]:
import pandas as pd
import numpy as np

import torch
from torch import nn
from torch.nn import functional as F

from sklearn import metrics
from sklearn import preprocessing

In [420]:
def Convert(lst):
    values = range(0, len(lst))
    res_dct = {lst[i]: values[i] for i in range(0, len(lst))}
    return res_dct

In [421]:
df = pd.read_csv('/Users/gafur/Desktop/train.data', delimiter = ',', names=[1,2,3,4,5,6,7])
df2 = pd.read_csv('/Users/gafur/Desktop/val.data', delimiter = ',', names=[1,2,3,4,5,6,7])

In [422]:
_replace = Convert(df[7].value_counts().index.tolist())
df = df.replace({7: _replace})
df2 = df2.replace({7: _replace})
_replace = Convert(df[3].value_counts().index.tolist())
df = df.replace({1: _replace, 3: _replace, 5: _replace})
df2 = df2.replace({1: _replace, 3: _replace, 5: _replace})

In [423]:
df

,1,2,3,4,5,6,7
0,7,1,7,5,2,5,0
1,6,2,1,7,3,8,6
2,5,1,2,7,0,7,1
3,6,2,7,1,1,5,1
4,7,1,7,7,1,2,2
...,...,...,...,...,...,...,...
25246,4,1,0,7,1,6,4
25247,6,1,1,5,6,4,0
25248,7,3,0,6,7,7,6
25249,4,1,2,5,1,2,1


In [481]:
X_train, y_train = df[[1, 2, 3, 4, 5, 6]].values, df[7].values
X_test, y_test = df2[[1, 2, 3, 4, 5, 6]].values, df2[7].values
X_train = np.squeeze(X_train)
X_test = np.squeeze(X_test)
X_train = torch.tensor(X_train, dtype = torch.float)
y_train = torch.tensor(y_train, dtype = torch.int64)
X_test = torch.tensor(X_test, dtype = torch.float)
y_test = torch.tensor(y_test, dtype = torch.int64)

In [503]:
class Perceptron(nn.Module):
    def __init__(self, n_hidden_layers, n_targets, n_rows):
        super(Perceptron, self).__init__()
        self.l1 = nn.Linear(n_rows, n_hidden_layers)
        self.l2 = nn.Linear(n_hidden_layers, n_hidden_layers)
        self.l22 = nn.Linear(n_hidden_layers, n_hidden_layers)
        self.l222 = nn.Linear(n_hidden_layers, n_hidden_layers)
        self.l3 = nn.Linear(n_hidden_layers, n_targets)

    def forward(self, x):
        x = F.tanh(self.l1(x))
        x = F.tanh(self.l2(x))
        x = F.relu(self.l22(x))
        x = F.relu(self.l222(x))
        return self.l3(x)

model = Perceptron(70, 18, 6)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.007)

def train():
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()

def predict():
    model.eval()
    test_loss = 0
    correct = 0

    output = model(X_test)

    test_loss += criterion(output, y_test).item()

    pred = output.data.max(1, keepdim=True)[1]
    correct += pred.eq(y_test.data.view_as(pred)).cpu().sum()

    test_loss /= len(X_test)
    print(f'===========================\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(X_test)} '
          f'({100. * correct / len(X_test):.0f}%)')
    return pred

def accuracy(y_test, y_hat):
    _sum = 0
    for i in range(len(y_test)):
        if y_test[i] == y_hat[i]:
            _sum += 1
    return _sum * 100 / len(y_test)
    
if __name__ == '__main__':
    for epoch in range(1000):
        train()
        y_pred = predict()
    print(accuracy(y_test, y_pred))


                


 0.0004, Accuracy: 1069/1403 (76%)
Test set: Average loss: 0.0004, Accuracy: 1073/1403 (76%)
Test set: Average loss: 0.0004, Accuracy: 1066/1403 (76%)
Test set: Average loss: 0.0004, Accuracy: 1073/1403 (76%)
Test set: Average loss: 0.0004, Accuracy: 1069/1403 (76%)
Test set: Average loss: 0.0004, Accuracy: 1073/1403 (76%)
Test set: Average loss: 0.0004, Accuracy: 1069/1403 (76%)
Test set: Average loss: 0.0004, Accuracy: 1073/1403 (76%)
Test set: Average loss: 0.0004, Accuracy: 1065/1403 (76%)
Test set: Average loss: 0.0004, Accuracy: 1070/1403 (76%)
Test set: Average loss: 0.0004, Accuracy: 1069/1403 (76%)
Test set: Average loss: 0.0004, Accuracy: 1061/1403 (76%)
Test set: Average loss: 0.0004, Accuracy: 1072/1403 (76%)
Test set: Average loss: 0.0004, Accuracy: 1060/1403 (76%)
Test set: Average loss: 0.0004, Accuracy: 1064/1403 (76%)
Test set: Average loss: 0.0004, Accuracy: 1053/1403 (75%)
Test set: Average loss: 0.0004, Accuracy: 1066/1403 (76%)
Test set: Average loss: 0.0004, Accur

In [492]:
_sum = 0
for i in range(len(y_pred)):
    if (y_pred[i] == y_test[i]):
        _sum += 1
print(_sum * 100 / len(y_test))

70.06414825374198
